#Time Series Forecasting 
###Sales Forecast Case Study





In [57]:
import pandas as pd
import numpy as np
%matplotlib inline

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [58]:
data = pd.read_csv('Sales_Transactions_Dataset_Weekly.csv')

In [59]:
data.shape

(811, 107)

In [60]:
data = data.filter(regex=r'Product|W')
data.head()

,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
0,P1,11,12,10,8,13,12,14,21,6,...,4,7,8,10,12,3,7,6,5,10
1,P2,7,6,3,2,7,1,6,3,3,...,2,4,5,1,1,4,5,1,6,0
2,P3,7,11,8,9,10,8,7,13,12,...,6,14,5,5,7,8,14,8,8,7
3,P4,12,8,13,5,9,6,9,13,13,...,9,10,3,4,6,8,14,8,7,8
4,P5,8,5,13,11,6,7,9,14,9,...,7,11,7,12,6,6,5,11,8,9


In [61]:
melt = data.melt(id_vars='Product_Code', var_name='Week', value_name='Sales')
melt.head(815)

,Product_Code,Week,Sales
0,P1,W0,11
1,P2,W0,7
2,P3,W0,7
3,P4,W0,12
4,P5,W0,8
...,...,...,...
810,P819,W0,0
811,P1,W1,12
812,P2,W1,6
813,P3,W1,11


In [62]:
melt['Product_Code'] = melt['Product_Code'].str.extract('(\d+)', expand=False).astype(int)
melt['Week'] = melt['Week'].str.extract('(\d+)', expand=False).astype(int)

melt = melt.sort_values(['Week', 'Product_Code'])
melt.head()

,Product_Code,Week,Sales
0,1,0,11
1,2,0,7
2,3,0,7
3,4,0,12
4,5,0,8


##Split the data

In [63]:
split_point = 40
##the past
melt_train = melt[melt['Week'] < split_point].copy()
##the future
melt_valid = melt[melt['Week'] >= split_point].copy()

In [64]:
melt_train.head()

,Product_Code,Week,Sales
0,1,0,11
1,2,0,7
2,3,0,7
3,4,0,12
4,5,0,8


In [65]:
melt_valid.head()

,Product_Code,Week,Sales
32440,1,40,7
32441,2,40,2
32442,3,40,3
32443,4,40,12
32444,5,40,5


##3. Set up a 1-step target

In [66]:
##sales_next_week for the first row take the value of sales of the second row .... 
melt_train['sales_next_week'] = melt_train.groupby("Product_Code")['Sales'].shift(-1)

In [67]:
melt_train[melt_train['Product_Code'] == 1].head(6)

,Product_Code,Week,Sales,sales_next_week
0,1,0,11,12.0
811,1,1,12,10.0
1622,1,2,10,8.0
2433,1,3,8,13.0
3244,1,4,13,12.0
4055,1,5,12,14.0


In [68]:
melt_valid['sales_next_week'] = melt_valid.groupby("Product_Code")['Sales'].shift(-1)

In [69]:
melt_train.tail(815)

,Product_Code,Week,Sales,sales_next_week
31625,816,38,4,6.0
31626,817,38,0,0.0
31627,818,38,0,0.0
31628,819,38,1,0.0
31629,1,39,17,NaN
...,...,...,...,...
32435,815,39,2,NaN
32436,816,39,6,NaN
32437,817,39,0,NaN
32438,818,39,0,NaN


In [70]:
##drop rows with NaN values an
melt_train = melt_train.dropna()

In [71]:
melt_train.tail(815)

,Product_Code,Week,Sales,sales_next_week
30814,816,37,2,4.0
30815,817,37,0,0.0
30816,818,37,0,0.0
30817,819,37,1,1.0
30818,1,38,12,17.0
...,...,...,...,...
31624,815,38,1,2.0
31625,816,38,4,6.0
31626,817,38,0,0.0
31627,818,38,0,0.0


##4. Create 4 Fundamental Features


###4.1 Lag
what if I don't have the last period? Get the closest available

In [72]:
##the lag_sales_1 for the second row take the value of sales of the first row
melt_train["lag_sales_1"] = melt_train.groupby("Product_Code")['Sales'].shift(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [73]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1
0,1,0,11,12.0,NaN
811,1,1,12,10.0,11.0
1622,1,2,10,8.0,12.0
2433,1,3,8,13.0,10.0
3244,1,4,13,12.0,8.0


In [74]:
melt_valid["lag_sales_1"] = melt_valid.groupby("Product_Code")['Sales'].shift(1)

##4.2 Difference

In [75]:
##sales - (lag_sales_1)
melt_train["diff_sales_1"] = melt_train.groupby("Product_Code")['Sales'].diff(1)

In [76]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1
0,1,0,11,12.0,NaN,NaN
811,1,1,12,10.0,11.0,1.0
1622,1,2,10,8.0,12.0,-2.0
2433,1,3,8,13.0,10.0,-2.0
3244,1,4,13,12.0,8.0,5.0


In [77]:
melt_valid["diff_sales_1"] = melt_valid.groupby("Product_Code")['Sales'].diff(1)

4.3 Rolling statistics
**Mean
**Max
**Min
**Std

In [78]:
##the mean of sales for the 4 rows
melt_train.groupby("Product_Code")['Sales'].rolling(4).mean()

Product_Code       
1             0          NaN
              811        NaN
              1622       NaN
              2433     10.25
              3244     10.75
                       ...  
819           28384     0.25
              29195     1.00
              30006     1.00
              30817     1.25
              31628     1.50
Name: Sales, Length: 31629, dtype: float64

In [79]:
##drop the level 0 of the index
melt_train.groupby("Product_Code")['Sales'].rolling(4).mean().reset_index(level=0, drop=True)

0          NaN
811        NaN
1622       NaN
2433     10.25
3244     10.75
         ...  
28384     0.25
29195     1.00
30006     1.00
30817     1.25
31628     1.50
Name: Sales, Length: 31629, dtype: float64

In [80]:
##drop the level 1 of the index
melt_train.groupby("Product_Code")['Sales'].rolling(4).mean().reset_index(level=1 , drop=True)

Product_Code
1        NaN
1        NaN
1        NaN
1      10.25
1      10.75
       ...  
819     0.25
819     1.00
819     1.00
819     1.25
819     1.50
Name: Sales, Length: 31629, dtype: float64

In [81]:
melt_train["mean_sales_4"] = melt_train.groupby("Product_Code")['Sales'].rolling(4).mean().reset_index(level=0, drop=True)

In [82]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4
0,1,0,11,12.0,NaN,NaN,NaN
811,1,1,12,10.0,11.0,1.0,NaN
1622,1,2,10,8.0,12.0,-2.0,NaN
2433,1,3,8,13.0,10.0,-2.0,10.25
3244,1,4,13,12.0,8.0,5.0,10.75


In [83]:
melt_valid["mean_sales_4"] = melt_valid.groupby("Product_Code")['Sales'].rolling(4).mean().reset_index(level=0, drop=True)

4.4 Date components (seasonality) - not available in this dataset
**Day of week
**Day of year
**Month
**Day of month

##5. Choose an evaluation metric

The mean absolute percentage error (MAPE):
*************************************************
MAPE=100/n*(somme(|T-P|/|T|))
************************************************
T:True Value
P:Predicted Value

problem(if T =0)
*************************************************
solution
*************************************************

 Weighted Mean Absolute Percentage Error (WMAPE):
****************************************************
WMAEP=somme(T-P)/somme(T)

In [84]:
def mape(y_true, y_pred):
    ape = np.abs((y_true - y_pred) / y_true)
    #ape[~np.isfinite(ape)] = 0. # VERY questionable
    ##if the True value = 0 so the result is no finite(maybe nan,infinite...) ==>replace with 1 ===> the erreor is 100% really bad
    ape[~np.isfinite(ape)] = 1. # pessimist estimate
    return np.mean(ape)



In [85]:
def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

##6. Establish baseline
Predict next week sales as equal to this week sales


In [86]:

y_pred = melt_train['Sales']
y_true = melt_train['sales_next_week']

In [87]:
mape(y_true, y_pred)

0.6721872645511404

In [88]:
wmape(y_true, y_pred)

0.30816465612331645

##7. Train the model

In [142]:
melt_train.head(5)

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4
0,1,0,11,12.0,NaN,NaN,NaN
1,2,0,7,6.0,NaN,NaN,NaN
2,3,0,7,11.0,NaN,NaN,NaN
3,4,0,12,8.0,NaN,NaN,NaN
4,5,0,8,5.0,NaN,NaN,NaN


In [143]:
features = ['Sales', 'lag_sales_1', 'diff_sales_1', 'mean_sales_4']

In [144]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from xgboost import XGBRegressor
imputer = SimpleImputer()




In [145]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', imputer ,features),
    ])


In [146]:
Xtr = melt_train[features]
ytr = melt_train['sales_next_week']

In [240]:
# mdl = XGBRegressor(n_estimators=1000, learning_rate=0.05, random_state=0,n_jobs=6)
##but it can't work with multiple colums as output so we work with random forest afterextending the model
mdl = RandomForestRegressor(n_estimators=1000, random_state=0)

In [241]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', mdl)
                             ])


In [242]:
Xtr

,Sales,lag_sales_1,diff_sales_1,mean_sales_4
0,11,NaN,NaN,NaN
1,7,NaN,NaN,NaN
2,7,NaN,NaN,NaN
3,12,NaN,NaN,NaN
4,8,NaN,NaN,NaN
...,...,...,...,...
31624,1,0.0,1.0,0.25
31625,4,2.0,2.0,3.25
31626,0,0.0,0.0,0.00
31627,0,0.0,0.0,0.00


In [243]:
ytr

0        12.0
1         6.0
2        11.0
3         8.0
4         5.0
         ... 
31624     2.0
31625     6.0
31626     0.0
31627     0.0
31628     0.0
Name: sales_next_week, Length: 31629, dtype: float64

In [244]:
my_pipeline.fit(Xtr, ytr)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Sales', 'lag_sales_1',
                                                   'diff_sales_1',
                                                   'mean_sales_4'])])),
                ('model', RandomForestRegressor(random_state=0))])

##8. Evaluate the model

In [245]:
melt_valid

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week
32440,1,40,7,11.0,NaN,NaN,NaN,4.0
32441,2,40,2,5.0,NaN,NaN,NaN,2.0
32442,3,40,3,5.0,NaN,NaN,NaN,6.0
32443,4,40,12,6.0,NaN,NaN,NaN,9.0
32444,5,40,5,8.0,NaN,NaN,NaN,7.0
...,...,...,...,...,...,...,...,...
42167,815,51,0,NaN,2.0,-2.0,0.50,NaN
42168,816,51,5,NaN,6.0,-1.0,5.25,NaN
42169,817,51,3,NaN,4.0,-1.0,1.75,NaN
42170,818,51,0,NaN,2.0,-2.0,0.50,NaN


In [246]:
xval = melt_valid[features]
yval = melt_valid['sales_next_week']

In [247]:
xval

,Sales,lag_sales_1,diff_sales_1,mean_sales_4
32440,7,NaN,NaN,NaN
32441,2,NaN,NaN,NaN
32442,3,NaN,NaN,NaN
32443,12,NaN,NaN,NaN
32444,5,NaN,NaN,NaN
...,...,...,...,...
42167,0,2.0,-2.0,0.50
42168,5,6.0,-1.0,5.25
42169,3,4.0,-1.0,1.75
42170,0,2.0,-2.0,0.50


In [248]:
yval

32440    11.0
32441     5.0
32442     5.0
32443     6.0
32444     8.0
         ... 
42167     NaN
42168     NaN
42169     NaN
42170     NaN
42171     NaN
Name: sales_next_week, Length: 9732, dtype: float64

In [249]:
preds = my_pipeline.predict(xval)

In [250]:
mape(yval, preds)

0.6463137461455405

In [251]:
wmape(yval, preds)

0.3004969729507602

##9. Extend the model to predict n-steps

In [252]:
melt_train['sales_next_next_week'] = melt_train.groupby("Product_Code")['Sales'].shift(-2)
melt_valid['sales_next_next_week'] = melt_valid.groupby("Product_Code")['Sales'].shift(-2)

In [253]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week
0,1,0,11,12.0,NaN,NaN,NaN,10.0
811,1,1,12,10.0,11.0,1.0,NaN,8.0
1622,1,2,10,8.0,12.0,-2.0,NaN,13.0
2433,1,3,8,13.0,10.0,-2.0,10.25,12.0
3244,1,4,13,12.0,8.0,5.0,10.75,14.0


In [254]:
melt_train = melt_train.dropna(subset=['sales_next_week','sales_next_next_week'])

In [255]:
Ytr1=melt_train[['sales_next_week', 'sales_next_next_week']]

In [256]:
Xtr1=melt_train[features]

In [258]:
my_pipeline.fit(Xtr1, Ytr1)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Sales', 'lag_sales_1',
                                                   'diff_sales_1',
                                                   'mean_sales_4'])])),
                ('model', RandomForestRegressor(random_state=0))])

In [260]:
xval1 = melt_valid[features]
yval1 = melt_valid[['sales_next_week', 'sales_next_next_week']]

In [261]:
preds = my_pipeline.predict(xval1)

In [262]:
mape(yval1, preds)

sales_next_week         0.650230
sales_next_next_week    0.682716
dtype: float64

In [263]:
wmape(yval1, preds)

sales_next_week         0.302810
sales_next_next_week    0.312631
dtype: float64

##10. Predicting new examples

In [270]:
melt_valid

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week
32440,1,40,7,11.0,NaN,NaN,NaN,4.0
32441,2,40,2,5.0,NaN,NaN,NaN,2.0
32442,3,40,3,5.0,NaN,NaN,NaN,6.0
32443,4,40,12,6.0,NaN,NaN,NaN,9.0
32444,5,40,5,8.0,NaN,NaN,NaN,7.0
...,...,...,...,...,...,...,...,...
42167,815,51,0,NaN,2.0,-2.0,0.50,NaN
42168,816,51,5,NaN,6.0,-1.0,5.25,NaN
42169,817,51,3,NaN,4.0,-1.0,1.75,NaN
42170,818,51,0,NaN,2.0,-2.0,0.50,NaN


In [271]:
new_examples = melt_valid[melt_valid['Week'] ==45].copy()
new_examples

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week
36495,1,45,10,12.0,8.0,2.0,7.25,3.0
36496,2,45,1,1.0,5.0,-4.0,3.00,4.0
36497,3,45,5,7.0,5.0,0.0,7.50,8.0
36498,4,45,4,6.0,3.0,1.0,6.50,8.0
36499,5,45,12,6.0,7.0,5.0,9.25,6.0
...,...,...,...,...,...,...,...,...
37301,815,45,0,0.0,1.0,-1.0,0.50,1.0
37302,816,45,2,4.0,4.0,-2.0,3.00,5.0
37303,817,45,2,2.0,0.0,2.0,1.00,0.0
37304,818,45,1,1.0,0.0,1.0,0.25,0.0


In [272]:
p = mdl.predict(new_examples[features])
p

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


array([[8.18514286, 3.65279762],
       [2.48345024, 3.9692735 ],
       [9.24592857, 6.58433333],
       ...,
       [0.95936643, 0.90709083],
       [0.2857096 , 0.31200404],
       [0.15358715, 0.15977923]])

In [273]:
new_examples['p_sales_next_week'] = p[:, 0]
new_examples['p_sales_next_next_week'] = p[:, 1]

In [274]:
new_examples.head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week,p_sales_next_week,p_sales_next_next_week
36495,1,45,10,12.0,8.0,2.0,7.25,3.0,8.185143,3.652798
36496,2,45,1,1.0,5.0,-4.0,3.00,4.0,2.483450,3.969274
36497,3,45,5,7.0,5.0,0.0,7.50,8.0,9.245929,6.584333
36498,4,45,4,6.0,3.0,1.0,6.50,8.0,7.963530,8.726576
36499,5,45,12,6.0,7.0,5.0,9.25,6.0,6.637976,14.110595


In [278]:
mape(new_examples['sales_next_week'], new_examples['p_sales_next_week'])

0.5966055916824173

In [279]:
mape(new_examples['sales_next_next_week'], new_examples['p_sales_next_next_week'])

0.5970310220149163

In [280]:
wmape(new_examples['sales_next_week'], new_examples['p_sales_next_week'])

0.28102359226961243

In [281]:
wmape(new_examples['sales_next_next_week'], new_examples['p_sales_next_next_week'])

0.2732464340284677